In [2]:
import os
from openai import OpenAI

# get your api key at https://app.nous.mesolitica.com/

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI(
    base_url="https://llm-router.nous.mesolitica.com",
)

In [3]:
import requests
import json

In [41]:
tools = [
    {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
]

In [42]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=1.0,
    max_tokens=1024,
    messages=[
    {
        "role": "user",
        "content": 'apa cuaca dkat sungai petani kedah right now'
    }
],
    tools=tools
)
r = json.loads(response.choices[0].message.content)
r

{'name': 'get_current_weather',
 'arguments': {'location': 'Sungai Petani, Kedah', 'unit': 'celsius'}}

And let say you have an API or a function to call,

In [43]:
def get_current_weather(location, unit="fahrenheit"):
    return {
        'location': location.upper(),
        'temperature': 38,
        'unit': unit
    }

In [44]:
result_function = globals().get(r['name'])(**r['arguments'])
result_function

{'location': 'SUNGAI PETANI, KEDAH', 'temperature': 38, 'unit': 'celsius'}

In [45]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=1.0,
    max_tokens=1024,
    messages=[
    {
        "role": "system",
        "content": f'Anda adalah chatbot yang berguna.'
    },
    {
        "role": "user",
        "content": f'teks `{result_function}` tukaq teks ke teks manusia natural'
    }
],
)
response.choices[0].message.content

'Di Sungai Petani, suhu adalah 38 darjah Celsius.'